Having trouble getting the gridcv to work with an AI model

In [1]:
# Import Libraries

import numpy as np
import pandas as pd
from matplotlib import pyplot
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_profiling as pp
import seaborn as sns
import os
import time
import zipfile
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import cohen_kappa_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import StandardScaler

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Conv1D
from keras.layers import BatchNormalization
from keras.layers import MaxPool1D
from keras.layers import Flatten
#from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers
from keras.optimizers import SGD
from keras.optimizers import RMSprop
from keras.optimizers import Adagrad
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
import os
colab = os.environ.get('COLAB_GPU', '10')
if (int(colab) == 0):
    from google.colab import drive
    drive.mount('/content/drive')
    #Path="drive/My Drive/Colab Notebooks/StockAnalysis"
    Path="" # ???
    DataPath="/content"
    RootPath="/root"    
else:
    #Path="/c/DataScience/Repo/Imbalanced_data"
    Path="/DataScience/Repo/Imbalanced_data"
    DataPath=Path
    RootPath="/Users/iowahawk89"

Load and prepare dataset

In [3]:
# Load the Data Set
df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')

In [4]:
X = df.loc[:, df.columns != 'Class']
y = df.loc[:, df.columns == 'Class']
X_train, X_test1, y_train, y_test1 = train_test_split(X,y, test_size = 0.4, random_state = 42)
X_test, X_val, y_test, y_val = train_test_split(X_test1,y_test1, test_size = 0.4, random_state = 42)
class_names=[0,1] 

In [5]:
# downsampling
yes = len(y_train[y_train['Class'] ==1]) 
yes_ind = y_train[y_train['Class'] == 1].index
no_ind = y_train[y_train['Class'] == 0].index
new_no_ind = np.random.choice(no_ind, yes, replace = False)
undersample_ind = np.concatenate([new_no_ind, yes_ind])
X_train = X_train.loc[undersample_ind]
y_train = y_train.loc[undersample_ind]

Create functions

In [6]:
class MyTimer():
    def __init__(self):
        self.start = time.time()
    def __enter__(self):
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        end = time.time()
        runtime = end - self.start
        msg = 'The function took {time} seconds to complete'
        print(msg.format(time=runtime))

In [7]:
def display_metrics(model_name, train_features, test_features, train_label, test_label, pred):
    try:
        print(model_name.score(test_features, test_label)) 
        print("Accuracy score (training): {0:.3f}".format(model_name.score(train_features, train_label))) 
        print("Accuracy score (validation): {0:.3f}".format(model_name.score(test_features, test_label))) 
    except Exception as e:
        print("error")  
    try:
        print(pd.Series(model_name.feature_importances_, index=train_features.columns[:]).nlargest(10).plot(kind='barh')) 
    except Exception as e:
        print("error")  

    print("Confusion Matrix:")
    tn, fp, fn, tp = confusion_matrix(test_label, pred).ravel()
    total = tn+ fp+ fn+ tp 
    print("false positive pct:",(fp/total)*100) 
    print("tn", " fp", " fn", " tp") 
    print(tn, fp, fn, tp) 
    print(confusion_matrix(test_label, pred)) 
    print("Classification Report") 
    print(classification_report(test_label, pred))
    print("Specificity =", tn/(tn+fp))
    print("Sensitivity =", tp/(tp+fn))
    return tn, fp, fn, tp

Explore Data

In [8]:
X_train

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
154340,101231.0,1.801432,-0.280307,-0.706453,1.409947,0.442907,1.067900,-0.323879,0.114636,2.047498,...,-0.025988,-0.617295,-1.416970,0.283564,-0.316434,-0.237016,-1.150447,0.014197,-0.032308,89.25
29392,35453.0,1.094547,-0.039137,1.404872,1.272242,-0.980858,0.012955,-0.620778,0.192990,0.537514,...,-0.094773,0.009329,0.221075,0.001036,0.547954,0.374995,-0.440598,0.072374,0.031356,9.99
158679,111638.0,2.031383,-0.116482,-3.184738,-0.604844,2.687599,3.010431,-0.373632,0.723440,0.439272,...,-0.116474,0.196974,0.608777,0.013515,0.637164,0.286998,-0.081198,0.015391,-0.036639,10.00
163866,116260.0,-0.429972,1.253972,-2.038432,-1.450138,3.756487,2.879112,0.638268,0.773257,-0.469925,...,-0.002523,0.198471,0.495190,-0.407864,0.484234,0.162100,-0.126988,0.042217,0.216694,1.00
65917,51803.0,-1.647527,1.093779,0.955744,1.578528,-0.407383,-0.215360,0.054097,0.783982,-0.944366,...,-0.168710,0.108617,0.336914,0.096226,0.463624,0.199963,-0.217167,-0.130682,-0.190706,23.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42674,41194.0,-7.896886,5.381020,-8.451162,7.963928,-7.862419,-2.376820,-11.949723,5.051356,-6.912076,...,0.645974,2.557944,0.926278,0.032795,0.638073,0.361887,0.444577,1.101923,0.205958,1.52
33276,37167.0,-7.923891,-5.198360,-3.000024,4.420666,2.272194,-3.394483,-5.283435,0.131619,0.658176,...,-2.169811,-0.734308,-0.599926,-4.908301,0.410170,-1.167660,0.520508,1.937421,-1.552593,12.31
182992,125612.0,1.889618,1.073099,-1.678018,4.173268,1.015516,-0.009389,-0.079706,0.064071,-0.714517,...,-0.153570,0.203728,0.733796,-0.036560,0.334306,0.147171,0.279556,0.031669,0.035883,3.22
154697,102625.0,-4.221221,2.871121,-5.888716,6.890952,-3.404894,-1.154394,-7.739928,2.851363,-2.507569,...,-0.227882,1.620591,1.567947,-0.578007,-0.059045,-1.829169,-0.072429,0.136734,-0.599848,7.59


In [9]:
nrows, ncols = X_train.shape # (602,30)
X_train_arr = X_train.to_numpy()
y_train_arr = y_train.to_numpy()
X_train_arr = X_train_arr.reshape(nrows, ncols, 1)

nrows, ncols = X_test.shape # (602,30)
X_test_arr = X_test.to_numpy()
y_test_arr = y_test.to_numpy()
X_test_arr = X_test_arr.reshape(nrows, ncols, 1)

In [10]:
input_shape = (ncols, 1)
print(input_shape)

(30, 1)


In [11]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

Using a 1D Convolution, which is useful for non-image data, such as time series related data (audio, sensor)
Be sure to remember to include an activation function in the initial Conv1D layer, otherwise the model will predict all minority case of 1's (FP rate is almost 100%) !!!

In [12]:
def create_cnn_model():
    model = Sequential()
    #model.add(Conv1D(32, 2, activation = 'relu', input_shape = input_shape))
    #model.add(Conv1D(filters=32, kernel_size=2, input_shape = (30) ))
    model.add(Conv1D(filters=32, kernel_size=10, strides=1, activation='swish', padding='valid', input_shape=input_shape ))
    # TypeError: 'int' object is not iterable
    model.add(BatchNormalization())
    model.add(MaxPool1D(2))
    model.add(Dropout(0.2))
    model.add(Conv1D(64, 2, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool1D(2))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    #model.compile(optimizer='adam', lr=0.001, loss = 'binary_crossentropy', metrics=['accuracy'])
    # or
    #opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)
    # Let's train the model using RMSprop
    model.compile(loss='binary_crossentropy',
                  #optimizer=opt,
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model


def create_model():
	# create model
	model = Sequential()
	model.add(Dense(12, input_dim=30, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	# Compile model
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [13]:
compiled_model = create_model()

compiled_model.fit(X_train, y_train)

model = KerasClassifier(build_fn=compiled_model, verbose=0)
#print(compiled_model.summary())
#model.fit(X_train, y_train)

X_train = X_train_arr.copy()
y_train = y_train_arr.copy()

gsc = GridSearchCV(
    estimator=model,
    param_grid={
        #'n_estimators': range(50,126,25),
        'epochs':[50,100]
        #'max_features': range(50,401,50),
        #'max_features': [50,100], # can be list or range or other
        #'criterion':['mse','mae']
        #min_samples_leaf': range(20,50,5),
        #min_samples_split': range(15,36,5),
    },
    scoring='roc_auc',
    #scoring='neg_mean_squared_error', # or look here for other choices 
    # https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
    cv=5
)


Epoch 1/1
602/602 [==============================] - 0s 464us/step - loss: 6230.0187 - accuracy: 0.5000


In [14]:
#model.fit(X_train, y_train)
print("Running GridSearchCV:")
with MyTimer():    
    grid_result = gsc.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

#for test_mean, train_mean, param in zip(
for test_mean, param in zip(
        grid_result.cv_results_['mean_test_score'],
        #grid_result.cv_results_['mean_train_score'],
        grid_result.cv_results_['params']):
    print("Test : %f with: %r" % (test_mean, param))
    
#model = ExtraTreesRegressor(**grid_result.best_params_)
model = Sequential(**grid_result.best_params_)

print("Fitting Model with GridSearch and Cross Validation:")
model.fit(X_train, y_train)

print(model.score(X_train, y_train))

Running GridSearchCV:


C:\Users\iowahawk89\miniconda3\envs\ds_tensorflow\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: __call__() missing 1 required positional argument: 'inputs'

  FitFailedWarning)


The function took 9.341293573379517 seconds to complete


TypeError: __call__() missing 1 required positional argument: 'inputs'

In [ ]:
epochs = 200 # maybe 200?
model = create_model()

print(model.summary())

#class_weight = {1: 0.75,
#                0: 0.25}
with MyTimer():

    #history = model.fit(X_train_arr, y_train_arr, epochs=epochs, validation_data=(X_test, y_test), verbose=1)
    history = model.fit(X_train_arr, y_train_arr, epochs=epochs, verbose=1, sample_weight=np.where(y_train_arr == 1,6.0,1.0).flatten())
    #history = model.fit(X_train_arr, y_train_arr, epochs=epochs, verbose=1, sample_weight=class_weight)
#plot_learningCurve(history, epochs)

In [ ]:
# predict probabilities for test set
yhat_probs = model.predict(X_test_arr, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(X_test_arr, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

tn, fp, fn, tp = display_metrics(model, X_train_arr, X_test_arr, y_train_arr, y_test_arr, yhat_classes)
#visualize(y_test_arr, yhat_classes, 'CNN Simple')

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_test_arr, yhat_classes, pos_label=1)
print('TEST | AUC Score: ' + str((metrics.auc(fpr, tpr))))

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset
dataset = numpy.loadtxt("http://nrvis.com/data/mldata/pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))